In [1]:
from bs4 import BeautifulSoup as soup

import requests
import time
import random

search_text = "golang middle"

In [2]:
import re

def wait ():
    sleep_time = {'min': 0.1, 'max': 0.5}
    time.sleep(sleep_time["min"]+(random.random()*(sleep_time["max"]-sleep_time["min"])))

url = "https://nn.hh.ru/search/vacancy"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:91.0) Firefox/91.0",
    "Accept": "*/*",
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive"
}

params = {
    "clusters": False,
    "ored_clusters": False,
    "enable_snippets": False,
    "salary": None,
    "st": "searchVacancy",
    "text": search_text,
    "page": 0
}

def get_max_page_count():
    
    response = requests.get(url, headers = headers, params = params)
    response_parsed = soup(response.text, 'html.parser')
    

    pager_buttons = response_parsed.select("div[data-qa='pager-block'] a[class='bloko-button'][data-qa='pager-page']")  
    
    max_page_number = 0
    
    for pager_button in (pager_buttons or []):
        number = int(pager_button.get_text())
        max_page_number = number if number > max_page_number else max_page_number
        
    wait()
    print("max page count: ",max_page_number)
    return max_page_number


vacancys_data = []
vacancys_on_page = False 
vacancys_page_count = get_max_page_count()

while params['page'] < vacancys_page_count:
    
    while not vacancys_on_page:
        
        response = requests.get(url, headers = headers, params = params)
        response_parsed = soup(response.text, 'html.parser')

        # ищем блоки с вакансиями и выдергиваем url
        vacancys_on_page = response_parsed.select("div[class=vacancy-serp-item]")  
        vacancys_on_page = (vacancys_on_page if len(vacancys_on_page or [])>0 else False)

        # если нашли вакансии - составляем под них структуры с данными
        if vacancys_on_page:
            for vacancy_block in vacancys_on_page:
                link_container = vacancy_block.select("a[class='bloko-link']")
                link_text = re.sub(r'\?.+',r'', str(link_container[0]['href']))
                vacancys_data.append({"url": link_text})

        print("page ", params['page'], ", count: ", len(vacancys_on_page or []) )
        
        # симулируем
        wait()
        
    vacancys_on_page = False

    params['page'] += 1


max page count:  6
page  0 , count:  20
page  1 , count:  20
page  2 , count:  20
page  3 , count:  20
page  4 , count:  20
page  5 , count:  10


In [23]:
#from deep_translator import GoogleTranslator
import re
import datetime

#translator = GoogleTranslator(source='ru', target='en')
    
def parse_date (raw_text):
    date_text = re.search(r'\d+\s+[^\d\s]+\s+\d+', raw_text or "")
    
    if date_text:
        date_text = date_text.group()

    date_splited = re.split("\s+", date_text)
    #translation = translator.translate(date_splited[-2])
    #translated_month_words = re.split("\s+", translation)
    #translation = translated_month_words[-1]
    
    a_month = ['', 'янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    month_abbr = re.search(r'^[\w+]{3}', date_splited[-2]).group()
    month_number = a_month.index(month_abbr)
    
    #try:
    #    datetime_object = datetime.datetime.strptime(translation, "%B")
    #except ValueError:
    #    return raw_text
    return datetime.date(int(date_splited[-1]), month_number, int(date_splited[0]))

In [24]:
from price_parser import Price

def process_salary(raw_salary):

    salary_value_strings = re.findall(r'((\d+\s*)+)+', raw_salary)
    salary_sequence = []
    for pair in salary_value_strings: salary_sequence.append(int(re.sub(r'\s+', r'', pair[0])))
    salary_sequence.sort()
    
    salary = {
        "from": salary_sequence[0] if len(salary_sequence)>0 else False,
        "to": salary_sequence[1] if len(salary_sequence)>1 else False,
        "gross": False if re.search(r'на\s+руки', raw_salary) else True,
        "currancy":  Price.fromstring(raw_salary).currency
    }

    return salary

In [5]:
import codecs

# разбираем страницу вакансии
counter = 1
for data in vacancys_data:
    
    response = requests.get(data['url'], headers = headers)
    response_parsed = soup(response.text, 'html.parser')    
   
    # вытаскиваем заголовок
    
    title = response_parsed.select("h1[data-qa = 'vacancy-title']")
    data["title"] = str(title[0].get_text()) if len(title)>0 else False
    
    # зарплата бля(
    
    salary_div = response_parsed.select("div[data-qa = 'vacancy-salary']")
    
    #print(counter," of ",len(vacancys_data)," : ", salary_span.get_text())
    counter += 1

    data["salary"] = str(salary_div[0].get_text()) if len(salary_div)>0 else False
    
    # название компании
    
    company_name_container = response_parsed.select("a[data-qa = 'vacancy-company-name'][class = 'vacancy-company-name'] > span[data-qa = 'bloko-header-2']")
    
    data["company_name"] = company_name_container[0].get_text() if len(company_name_container)>0 else False
    
    # проверенный работодатель
    
    verified_company_link = response_parsed.select("div[class = 'vacancy-company-badge'] > a[class = 'bloko-link'][href = 'https://feedback.hh.ru/article/details/id/5951']")
    data["company_verified"] = True if len(verified_company_link)>=1 else False
    
    # адрес 
    
    full_adress = response_parsed.select("span[data-qa = 'vacancy-view-raw-address']") 
    city_or_smth = response_parsed.select("p[data-qa = 'vacancy-view-location']")
    
    city_or_smth = city_or_smth[0] if len(city_or_smth)>0 else False
    full_adress = full_adress[0] if len(full_adress)>0 else False
        
    data["adress"] = ""
    
    if city_or_smth or full_adress:
        data["adress"] = (city_or_smth or full_adress).get_text()
        
    # режим на объекте
    employment_mode = response_parsed.select("p[data-qa = 'vacancy-view-employment-mode']") 
    data["employment_mode"] = employment_mode[0].get_text() if len(employment_mode)>0 else False
    
    # опыт работы 
    experience = response_parsed.select("span[data-qa = 'vacancy-experience']") 
    data["experience"] = experience[0].get_text() if len(experience)>0 else False
    
    # parttime
    parttime = response_parsed.select("p[data-qa = 'vacancy-view-parttime-options']") 
    data["parttime"] = parttime[0].get_text() if len(parttime)>0 else False

    # accept-temporary
    accept_temporary = response_parsed.select("p[data-qa = 'vacancy-view-accept-temporary']") 
    data["accept_temporary"] = accept_temporary[0].get_text() if len(accept_temporary)>0 else False

    # full_description
    description = response_parsed.select("div[class = 'vacancy-section']") 
    data["description"] = description[0].get_text() if len(description)>0 else False
  
    # дата
    date = response_parsed.select("p[class = 'vacancy-creation-time']") 
    data["creation-time"] = str(date[0].get_text()) if len(date)>0 else False
    
    # тэги 
    tags = response_parsed.select("div[class = 'bloko-tag-list'] > div") 
    tags = tags[0] if len(tags)>0 else False
    
    tag_list = []
    if tags:
        for tag in tags.children:
            tag_list.append(tag.get_text())
    
    data["tag"] = tag_list
    
    # на скорую руку подчищаем вывод
    for key in data:
        if type(key) == type(""):
            data[key] = codecs.encode(str(data[key]),'utf8')
            data[key] = codecs.decode(data[key],'utf8')
            
    print(counter,"of",len(vacancys_data))
    
    # симулируем
    
    wait()    


2 of 110
3 of 110
4 of 110
5 of 110
6 of 110
7 of 110
8 of 110
9 of 110
10 of 110
11 of 110
12 of 110
13 of 110
14 of 110
15 of 110
16 of 110
17 of 110
18 of 110
19 of 110
20 of 110
21 of 110
22 of 110
23 of 110
24 of 110
25 of 110
26 of 110
27 of 110
28 of 110
29 of 110
30 of 110
31 of 110
32 of 110
33 of 110
34 of 110
35 of 110
36 of 110
37 of 110
38 of 110
39 of 110
40 of 110
41 of 110
42 of 110
43 of 110
44 of 110
45 of 110
46 of 110
47 of 110
48 of 110
49 of 110
50 of 110
51 of 110
52 of 110
53 of 110
54 of 110
55 of 110
56 of 110
57 of 110
58 of 110
59 of 110
60 of 110
61 of 110
62 of 110
63 of 110
64 of 110
65 of 110
66 of 110
67 of 110
68 of 110
69 of 110
70 of 110
71 of 110
72 of 110
73 of 110
74 of 110
75 of 110
76 of 110
77 of 110
78 of 110
79 of 110
80 of 110
81 of 110
82 of 110
83 of 110
84 of 110
85 of 110
86 of 110
87 of 110
88 of 110
89 of 110
90 of 110
91 of 110
92 of 110
93 of 110
94 of 110
95 of 110
96 of 110
97 of 110
98 of 110
99 of 110
100 of 110
101 of 110
102 of

In [6]:
import pandas as pd

df = pd.DataFrame(columns= list(vacancys_data[0].keys()))

ind = 0
for data in vacancys_data:     
    if (len(df.columns)==len(data)):
        df.loc[ind] = data
        ind+=1

df.to_csv(search_text+".csv", encoding="utf8" )

In [25]:
import pandas as pd
df = pd.DataFrame()
#search_text="golang middle"
df=pd.read_csv(search_text+".csv", encoding="utf8" )
df

,Unnamed: 0,url,title,salary,company_name,company_verified,adress,employment_mode,experience,parttime,accept_temporary,description,creation-time,tag
0,0,https://nn.hh.ru/vacancy/50977645,Blockchain разработчик/Frontend-разработчик/Fu...,от 350 000 до 550 000 руб. на руки,ArtVolkov.ru,False,"Нагатинская, Москва , Нагатинская улица, 1","Полная занятость, полный день",1–3 года,False,"Возможно временное оформление: договор услуг, ...",xDAO — конструктор для создания и управления д...,Вакансия опубликована 24 января 2022 в Москве,['JavaScript']
1,1,https://nn.hh.ru/vacancy/50939243,Middle Go / Golang разработчик,от 190 000 руб. на руки,ООО ФАЙВДЖЕН,False,"Москва, Усачёва улица, 33с2","Полная занятость, полный день",1–3 года,False,False,Привет! Мы занимаемся разработкой сложных и ин...,Вакансия опубликована 23 января 2022 в Москве,['Git']
2,2,https://nn.hh.ru/vacancy/50835097,Middle/Senior Golang developer,от 200 000 до 300 000 руб. на руки,Pushwoosh,False,"Москва, Пресненская набережная, 6с2","Полная занятость, полный день",3–6 лет,False,False,Мы - Pushwoosh – глобальный хайлоад продукт дл...,Вакансия опубликована 22 января 2022 в Москве,['PostgreSQL']
3,3,https://nn.hh.ru/vacancy/50009011,Backend / Developer Middle Golang,от 170 000 руб. до вычета налогов,ООО ВФМ технолоджи,False,"Москва, Тверская улица, 16с3","Полная занятость, полный день",1–3 года,False,False,"Приглашаем бекенд-разработчикa Golang , которы...",Вакансия опубликована 22 января 2022 в Москве,['Git']
4,4,https://nn.hh.ru/vacancy/51160195,Python Developer,от 200 000 до 350 000 руб. на руки,ООО АСД Технолоджиз,False,Санкт-Петербург,"Полная занятость, полный день",1–3 года,False,False,"АСД Технолоджиз, разработчик платформы cloudik...",Вакансия опубликована 23 января 2022 в Санкт-П...,[]
5,5,https://nn.hh.ru/vacancy/50825858,Golang разработчик,от 130 000 до 220 000 руб. на руки,Kvando Technologies,False,Москва,"Полная занятость, удаленная работа",1–3 года,False,"Возможно временное оформление: договор услуг, ...",KVANDO Technologies ЗП: 130 000 - 220 000 р. K...,Вакансия опубликована 23 января 2022 в Москве,['PostgreSQL']
6,6,https://nn.hh.ru/vacancy/50789953,Lead/Senior/Middle Rust Developer,от 4 000 до 12 000 USD на руки,Algalon Capital,False,Москва,"Полная занятость, полный день",1–3 года,False,False,Algalon Capital is looking to hire a Lead/Seni...,Вакансия опубликована 22 января 2022 в Москве,['Blockchain']
7,7,https://nn.hh.ru/vacancy/51107108,Middle Golang developer / Программист Golang,от 200 000 руб. на руки,Lenvendo,False,"Старая Деревня, Санкт-Петербург, улица Савушки...","Полная занятость, полный день",3–6 лет,False,False,\r\n\r\n\r\n\r\n\r\nРазрабатываем и поддержива...,Вакансия опубликована 23 января 2022 в Санкт-П...,['Golang']
8,8,https://nn.hh.ru/vacancy/50737128,Golang developer (middle),от 180 000 руб. на руки,ООО ПРОФИТКЛИКС,False,"Краснодар, улица Красных Партизан, 367","Полная занятость, полный день",1–3 года,False,False,"Привет! Мы - ProfitClicks, продуктовая компани...",Вакансия опубликована 21 января 2022 в Москве,['Golang']
9,9,https://nn.hh.ru/vacancy/42992992,Golang Middle developer,от 2 500 до 3 000 USD на руки,B2Broker Киев,False,Киев,"Полная занятость, удаленная работа",1–3 года,False,False,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nМы — команда м...,Вакансия опубликована 21 января 2022 в Киеве,['SQL']


In [26]:
from forex_python.converter import CurrencyRates
import forex_python.converter as fpc
import math
cr = CurrencyRates()

for i in range(len(df.index)):

    if type(df.at[i,"creation-time"]) == str:
        df.at[i,"creation-time"] = parse_date(df.at[i,"creation-time"])
    if type(df.at[i,"salary"]) == str:
        salary = process_salary(df.at[i,"salary"])
        
        salary_value = False
        
        if salary["from"] and salary["to"]:
            salary_value = ((salary["from"] or 0) + (salary["to"] or 0))/2
        else:
            salary_value = (salary["from"] or salary["to"]) or False
            
        if salary_value:
            if salary["currancy"]:
                if not(re.search(r'руб.+', salary["currancy"])):
                    try:
                        salary_value = math.floor(cr.convert(salary["currancy"], 'RUB', salary_value))
                        print(salary_value, salary["currancy"])                       
                    except fpc.RatesNotAvailableError:
                        print("Unexpected currancy "+salary["currancy"])
            df.at[i,"salary"] = salary_value

612181 USD
210437 USD
191306 USD
Unexpected currancy KZT
478267 USD
191306 USD
260513 EUR
Unexpected currancy KZT


In [40]:
import pandas as pd

pd.set_option("display.max_rows",None)
display(df)


,Unnamed: 0,url,title,salary,company_name,company_verified,adress,employment_mode,experience,parttime,accept_temporary,description,creation-time,tag
0,0,https://nn.hh.ru/vacancy/50977645,Blockchain разработчик/Frontend-разработчик/Fu...,450000,ArtVolkov.ru,False,"Нагатинская, Москва , Нагатинская улица, 1","Полная занятость, полный день",1–3 года,False,"Возможно временное оформление: договор услуг, ...",xDAO — конструктор для создания и управления д...,2022-01-24,['JavaScript']
1,1,https://nn.hh.ru/vacancy/50939243,Middle Go / Golang разработчик,190000,ООО ФАЙВДЖЕН,False,"Москва, Усачёва улица, 33с2","Полная занятость, полный день",1–3 года,False,False,Привет! Мы занимаемся разработкой сложных и ин...,2022-01-23,['Git']
2,2,https://nn.hh.ru/vacancy/50835097,Middle/Senior Golang developer,250000,Pushwoosh,False,"Москва, Пресненская набережная, 6с2","Полная занятость, полный день",3–6 лет,False,False,Мы - Pushwoosh – глобальный хайлоад продукт дл...,2022-01-22,['PostgreSQL']
3,3,https://nn.hh.ru/vacancy/50009011,Backend / Developer Middle Golang,170000,ООО ВФМ технолоджи,False,"Москва, Тверская улица, 16с3","Полная занятость, полный день",1–3 года,False,False,"Приглашаем бекенд-разработчикa Golang , которы...",2022-01-22,['Git']
4,4,https://nn.hh.ru/vacancy/51160195,Python Developer,275000,ООО АСД Технолоджиз,False,Санкт-Петербург,"Полная занятость, полный день",1–3 года,False,False,"АСД Технолоджиз, разработчик платформы cloudik...",2022-01-23,[]
5,5,https://nn.hh.ru/vacancy/50825858,Golang разработчик,175000,Kvando Technologies,False,Москва,"Полная занятость, удаленная работа",1–3 года,False,"Возможно временное оформление: договор услуг, ...",KVANDO Technologies ЗП: 130 000 - 220 000 р. K...,2022-01-23,['PostgreSQL']
6,6,https://nn.hh.ru/vacancy/50789953,Lead/Senior/Middle Rust Developer,612181,Algalon Capital,False,Москва,"Полная занятость, полный день",1–3 года,False,False,Algalon Capital is looking to hire a Lead/Seni...,2022-01-22,['Blockchain']
7,7,https://nn.hh.ru/vacancy/51107108,Middle Golang developer / Программист Golang,200000,Lenvendo,False,"Старая Деревня, Санкт-Петербург, улица Савушки...","Полная занятость, полный день",3–6 лет,False,False,\r\n\r\n\r\n\r\n\r\nРазрабатываем и поддержива...,2022-01-23,['Golang']
8,8,https://nn.hh.ru/vacancy/50737128,Golang developer (middle),180000,ООО ПРОФИТКЛИКС,False,"Краснодар, улица Красных Партизан, 367","Полная занятость, полный день",1–3 года,False,False,"Привет! Мы - ProfitClicks, продуктовая компани...",2022-01-21,['Golang']
9,9,https://nn.hh.ru/vacancy/42992992,Golang Middle developer,210437,B2Broker Киев,False,Киев,"Полная занятость, удаленная работа",1–3 года,False,False,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nМы — команда м...,2022-01-21,['SQL']


In [ ]:
from collections import Counter
import scipy

vacancy_names = df.tag # change here to change source of data/words etc


descriptions = df.description

import re

additional_tegs = []

for dsc in descriptions:
    searched = re.findall(r"(([A-z]+[\s/-]*)+)+", dsc)
    for pair in searched: additional_tegs.append(list(pair)[0])

#print(additional_tegs)        
    
all_tags = []
for tag in vacancy_names:
    all_tags.append("".join(tag)) 



from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
cloud = ''
for x in list(additional_tegs):
    cloud+=x+' '

wordcloud = WordCloud(width = 800, height = 800, 

                stopwords = stopwords, 
                min_font_size = 8,background_color='white'
                     ).generate(cloud)

import matplotlib.pylab as plt
plt.figure(figsize = (14, 14)) 
plt.imshow(wordcloud)
plt.savefig('vacancy_cloud.png')


teg_frequencys = Counter(additional_tegs)
teg_frequencys = dict(teg_frequencys)

new_series = pd.Series(data=list(teg_frequencys), index=list(teg_frequencys.values()))
new_series=new_series.sort_index(ascending=False)

freq = dict(Counter(list(new_series.index)))
freq_values = list(freq.values())
freq_values.sort(reverse=True)

freq_median = scipy.median(freq_values)

new_index = []
for ind in list(freq):     
        if freq[ind]>scipy.mean(freq_values):
            print(freq[ind],scipy.mean(freq_values))
            new_index.append(ind)

          
        
#print(freq,"median",freq_values,freq_median,"mean",scipy.mean(freq_values),new_index)
new_series=new_series.drop(new_index)
#print(new_series)

from matplotlib import pyplot as plt

plt.figure()
plt.xticks(rotation='vertical')
plt.bar(new_series.values, new_series.index)
plt.show()


patches, texts = plt.pie(new_series.index, labels = new_series.values)
plt.legend(patches, labels, loc="best")
plt.show()

In [148]:
! git add *
! git commit -m "fixed salary parsing"
! git push 

The file will have its original line endings in your working directory
The file will have its original line endings in your working directory


[main b5fb745] fixed salary parsing
 6 files changed, 11800 insertions(+), 18931 deletions(-)
 rewrite .ipynb_checkpoints/main-checkpoint.ipynb (69%)
 create mode 100644 golang middle.csv
 create mode 100644 golang.csv
 rewrite lua.csv (80%)
 rewrite main.ipynb (84%)
 rewrite vacancy_cloud.png (95%)


remote: This repository moved. Please use the new location:        
remote:   https://github.com/LuaSavage/hh_ru_parser.git        
To https://github.com/russianSlave/hh_ru_parser.git
   e3b9e60..b5fb745  main -> main
